In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_21124/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)


    def keep_random_models(self, n_models=3):
        # Extract block part from 'model' column
        self.results_df['block'] = self.results_df['model'].apply(lambda x: '_'.join(x.split('_')[1:3]))
        
        # Group by 'block' and keep n_models randomly from each group
        def select_random_models(group):
            return group.sample(n=min(n_models, len(group)), random_state=1)
        
        # Apply the function and ensure grouping columns are excluded from the final DataFrame
        grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)
        self.results_df = grouped.reset_index(drop=True).drop(columns=['block'])

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.keep_random_models()
        self.results_df = self.results_df.sort_values(by="r2_sup", ascending=False, ignore_index=True)
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



/tmp/ipykernel_21124/3323087823.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)


,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_26_8,0.999857,0.830434,0.822657,0.999785,0.000594,1.133885,0.634345,0.001202,0.081825,0.024379,1.009138,0.024763,144.856110,286.191283,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
1,model_1_25_7,0.999818,0.830399,0.823079,0.998994,0.000757,1.134125,0.632836,0.003299,0.083878,0.027522,1.011646,0.027955,144.371114,285.706287,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
2,model_1_26_3,0.999820,0.830305,0.823735,0.999721,0.000751,1.134752,0.630488,0.001558,0.086722,0.027396,1.011540,0.027828,144.389367,285.724540,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
3,model_1_25_1,0.999873,0.830230,0.822440,0.999532,0.000527,1.135250,0.635123,0.001533,0.074864,0.022960,1.008105,0.023322,145.096005,286.431177,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
4,model_1_26_9,0.999864,0.829687,0.823566,0.999791,0.000568,1.138882,0.631096,0.001164,0.076517,0.023833,1.008733,0.024208,144.946768,286.281940,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,model_1_1_7,0.907227,0.740065,0.705440,0.900940,0.386171,1.738183,1.053623,0.281639,1.349992,0.621427,6.937488,0.631214,131.902949,273.238122,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
144,model_1_2_0,0.915605,0.739948,0.734970,0.931732,0.351297,1.738972,0.947999,0.317118,1.237097,0.592703,6.401283,0.602037,132.092249,273.427421,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
145,model_1_1_3,0.898403,0.721927,0.711870,0.890734,0.422901,1.859475,1.030623,0.310655,1.280763,0.650308,7.502215,0.660550,131.721236,273.056408,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
146,model_1_1_0,0.884942,0.711336,0.685920,0.881328,0.478930,1.930294,1.123444,0.337399,1.348082,0.692048,8.363685,0.702947,131.472401,272.807573,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
